In [2]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install openpyxl
import numpy as np
import pandas as pd
flowdata = pd.read_excel(r'./flowdata.xlsx', sheet_name='Scenario1')

**Equation**: (E1)(KTB) + (E2)(KTC) - (E3)(SKB) - (E4)(SKC) = 0
- Each set of flow values for KTB, KTC, SKB, SKC can be geometrically represented as a line in 4-dimensional space defined by the variables E1, E2, E3, E4
- By iterating through test_values for E1, E2, E3, E4, we find the point X that minimises the sum of the distance X and all the lines
- Find num_results number of points X that provide the num_results lowest sum of distances

In [10]:
num_results = 10
results_array = [[[0, 0, 0, 0], sys.float_info.max] for _ in range(num_results)]
results = pd.DataFrame(results_array, index=range(1,11))

In [11]:
results

,0,1
1,"[0, 0, 0, 0]",1.797693e+308
2,"[0, 0, 0, 0]",1.797693e+308
3,"[0, 0, 0, 0]",1.797693e+308
4,"[0, 0, 0, 0]",1.797693e+308
5,"[0, 0, 0, 0]",1.797693e+308
6,"[0, 0, 0, 0]",1.797693e+308
7,"[0, 0, 0, 0]",1.797693e+308
8,"[0, 0, 0, 0]",1.797693e+308
9,"[0, 0, 0, 0]",1.797693e+308
10,"[0, 0, 0, 0]",1.797693e+308
